In [29]:
import pyblp
import numpy as np
import pandas as pd

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'0.12.0'

In [67]:
# Load the data 
product_data = pd.read_csv(pyblp.data.NEVO_PRODUCTS_LOCATION)
product_data.head(10)

,market_ids,city_ids,quarter,product_ids,firm_ids,brand_ids,shares,prices,sugar,mushy,...,demand_instruments10,demand_instruments11,demand_instruments12,demand_instruments13,demand_instruments14,demand_instruments15,demand_instruments16,demand_instruments17,demand_instruments18,demand_instruments19
0,C01Q1,1,1,F1B04,1,4,0.012417,0.072088,2,1,...,2.116358,-0.154708,-0.005796,0.014538,0.126244,0.067345,0.068423,0.034800,0.126346,0.035484
1,C01Q1,1,1,F1B06,1,6,0.007809,0.114178,18,1,...,-7.374091,-0.576412,0.012991,0.076143,0.029736,0.087867,0.110501,0.087784,0.049872,0.072579
2,C01Q1,1,1,F1B07,1,7,0.012995,0.132391,4,1,...,2.187872,-0.207346,0.003509,0.091781,0.163773,0.111881,0.108226,0.086439,0.122347,0.101842
3,C01Q1,1,1,F1B09,1,9,0.005770,0.130344,3,0,...,2.704576,0.040748,-0.003724,0.094732,0.135274,0.088090,0.101767,0.101777,0.110741,0.104332
4,C01Q1,1,1,F1B11,1,11,0.017934,0.154823,12,0,...,1.261242,0.034836,-0.000568,0.102451,0.130640,0.084818,0.101075,0.125169,0.133464,0.121111
5,C01Q1,1,1,F1B13,1,13,0.026602,0.137049,14,0,...,0.337554,0.023510,0.000264,0.086280,0.072336,0.022251,0.105644,0.116037,0.099651,0.105727
6,C01Q1,1,1,F1B17,1,17,0.025015,0.144209,3,1,...,2.617504,-0.195578,0.004489,0.094150,0.138474,0.110273,0.101192,0.106082,0.143585,0.120973
7,C01Q1,1,1,F1B30,1,30,0.005058,0.128191,4,0,...,2.591142,0.044275,-0.004563,0.108831,0.135491,0.128176,0.059036,0.085440,0.044623,0.097111
8,C01Q1,1,1,F1B45,1,45,0.005332,0.149611,14,0,...,0.489811,0.026016,-0.000066,0.114297,0.116368,0.141625,0.095104,0.122102,0.131221,0.119009
9,C01Q1,1,1,F2B05,2,5,0.038068,0.108514,1,0,...,2.727929,0.035499,-0.007844,0.083079,0.020242,-0.020562,0.064436,0.082678,-0.007339,0.072053


In [68]:
# Show the list of columns
product_data.columns

Index(['market_ids', 'city_ids', 'quarter', 'product_ids', 'firm_ids',
       'brand_ids', 'shares', 'prices', 'sugar', 'mushy',
       'demand_instruments0', 'demand_instruments1', 'demand_instruments2',
       'demand_instruments3', 'demand_instruments4', 'demand_instruments5',
       'demand_instruments6', 'demand_instruments7', 'demand_instruments8',
       'demand_instruments9', 'demand_instruments10', 'demand_instruments11',
       'demand_instruments12', 'demand_instruments13', 'demand_instruments14',
       'demand_instruments15', 'demand_instruments16', 'demand_instruments17',
       'demand_instruments18', 'demand_instruments19'],
      dtype='object')

# Formulate the problem and solve using MNL

In [69]:
# Setup the formulation using the linear demand side variables
mnlogit_formulation = pyblp.Formulation('prices', absorb='C(product_ids)')
mnlogit_formulation

prices + Absorb[C(product_ids)]

### What does `absorb` do exactly?
- Without using the `absorb` keyword in categorical columns that has constant effect (i.e. the effect does not change over time or changes extremely slowly), the categorical columns will be turned into **IV (Instrument Variables)** a.k.a. Expanded as dummy features added into the feature columns of the **Formulation**. `absorb` maintains them as categorical codes instead to avoid running to the problem of curse of dimensionality.
- Using `absorb` also disables addition of *intercept* by default.

In [70]:
# Set up the problem from the formulation
mnl_problem = pyblp.Problem(mnlogit_formulation, product_data)
mnl_problem

Dimensions:
 T    N     F    K1    MD    ED 
---  ----  ---  ----  ----  ----
94   2256   5    1     20    1  

Formulations:
     Column Indices:          0   
--------------------------  ------
X1: Linear Characteristics  prices

In [71]:
# Solve the problem
mnl_logit_results = mnl_problem.solve()
mnl_logit_results

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix
Step    Value    Shares   Condition Number
----  ---------  -------  ----------------
 2    +1.9E+02      0         +5.7E+07    

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
  prices  
----------
 -3.0E+01 
(+1.0E+00)

In [47]:
mnl_logit_results.converged

True

# Set up a Nested Logit Example
## Including `nesting_ids` which is a reserved word automatically trigers nested logit computation.
### Two options are explored:
- Group all the items in one group and the second nest will be outside good since it is normalized with respect to *outside good*
- Create nests using a particular column in product data. In this example, `mushy` column which has two groups is used. This means two nests. Since `mushy` is collinear with `product_ids`, the `absorb` product effects is removed in the configuration of the **Formulation**. The `absorb` option cannot be used when there is collinearity.
### Key IVs to include:
- Instrument for $\rho S_{j|h(j)t}$ i.e. the share of product $j$ in group $h$ where $\rho$ is the _nest parameter_ (a measure of correlation between products in a nest).
- For varying number of products per nest, we also need to instrument for the _number of products per nest_. For this example, number of products per nest does not vary and needs not be instrumented for.

In [60]:
def nl_solver(data:pd.DataFrame):
    groups = data.groupby(['market_ids', 'nesting_ids'])
    data['demand_instruments20'] = groups['shares'].transform(np.size)
    nl_formulation = pyblp.Formulation('0 + prices')
    problem = pyblp.Problem(nl_formulation, data)
    return problem.solve(rho=0.7)

In [61]:
# Copy the product data and solve for one nest only and then switch to multiple nests.
nl_data1 = product_data.copy()
nl_data1["nesting_ids"] = 1
nl_results1 = nl_solver(nl_data1)
nl_results1

Problem Results Summary:
GMM   Objective    Projected    Reduced   Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Hessian   Shares   Condition Number  Condition Number 
----  ---------  -------------  --------  -------  ----------------  -----------------
 2    +2.0E+02     +1.7E-09     +1.1E+04     0         +2.0E+09          +3.0E+04     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective 
   Time      Converged   Iterations   Evaluations
-----------  ---------  ------------  -----------
 00:00:01       Yes          3             8     

Rho Estimates (Robust SEs in Parentheses):
All Groups
----------
 +9.8E-01 
(+1.4E-02)

Beta Estimates (Robust SEs in Parentheses):
  prices  
----------
 -1.2E+00 
(+4.0E-01)

## $\rho$ has cleverly become a parameter to be estimated. This means that the initial $\rho$ was just a starting guess. The algorithm discovers the nesting relationships and estimates the best value for $\rho$. 

In [62]:
nl_results1.problem

Dimensions:
 T    N     F    K1    MD    H 
---  ----  ---  ----  ----  ---
94   2256   5    1     21    1 

Formulations:
     Column Indices:          0   
--------------------------  ------
X1: Linear Characteristics  prices

### Next, let us solve by using a column to perform the nesting. `mushy` has two nests.

In [64]:
df2 = product_data.copy()
df2['nesting_ids'] = df2['mushy']
nl_results2 = nl_solver(df2)
nl_results2

Problem Results Summary:
GMM   Objective    Projected    Reduced   Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Hessian   Shares   Condition Number  Condition Number 
----  ---------  -------------  --------  -------  ----------------  -----------------
 2    +6.9E+02     +5.1E-10     +5.6E+03     0         +5.1E+08          +2.0E+04     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective 
   Time      Converged   Iterations   Evaluations
-----------  ---------  ------------  -----------
 00:00:01       Yes          3             8     

Rho Estimates (Robust SEs in Parentheses):
All Groups
----------
 +8.9E-01 
(+1.9E-02)

Beta Estimates (Robust SEs in Parentheses):
  prices  
----------
 -7.8E+00 
(+4.8E-01)

In [65]:
nl_results2.problem

Dimensions:
 T    N     F    K1    MD    H 
---  ----  ---  ----  ----  ---
94   2256   5    1     21    2 

Formulations:
     Column Indices:          0   
--------------------------  ------
X1: Linear Characteristics  prices

### What is the impact of the adjusted `nesting parameter` on prices parameter estimate when we have a single and double nest?

In [66]:
print(f"Single nest: {nl_results1.beta[0] / (1 - nl_results1.rho)}")
print(f"Two nests: {nl_results2.beta[0] / (1 - nl_results2.rho)}")

Single nest: [[-67.39338888]]
Two nests: [[-72.27074638]]
